# Code for Ice-Cube 3D CNN
Using Wahid's tutorial keras code on ICE-Cube CNN data.
- October 5, 2018: 2D CNN works.
- October 6, 2018: got simple 3D CNN to work.
- Oct 10, 2018: Changes made:
    - Using cross validation and shuffling arguments of the fit function.
- Oct 16, 2018: Reading in actual data, which is very large. Saving to files. Split code into 2 parts: process data and train. Putting back shuffle and test options.
- Oct 18, 2018: This notebook does the entire workflow (except processing raw data) for 4 different models, in modular way, without classes.
The data formatting is still not modular.
- Oct 22, 2018: Saving model and history to project space. history files are 12GB! Just saving history.history to save space.
- Oct 24, 2018: Adding weights to testing process.
- Oct 29, 2018: Saving predicted values now, for quick plotting. Train-cv on regular data, Testing on reserved data.

In [3]:
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import glob
import pickle
import time

In [4]:
%matplotlib widget
# %matplotlib inline

Useful blog for keras conv3D: http://learnandshare645.blogspot.com/2016/06/3d-cnn-in-keras-action-recognition.html

In [5]:
# keras modules
import keras
from keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.models import load_model



Using TensorFlow backend.


### Steps:
- Data processing
    - Read raw data
    - Process data
    - Save process data
    - Read processed data
- Model
    - Define model
    - Train model
    - Validate model
    - Plot accuracy and loss
    - Save model
- Test
    - Read model
    - Read training data
    - Get weights
    - Test model
    - Plot ROC curve


### Process data 
#### Get data from the .hdf5 files. This takes a long time. Do this only once.



Run this is in bash


/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/extracted_data_v/scripts/run_extract.sh

## Modules

### Extracting data

In [6]:
# Load data from files
def f_load_data(data_dir,f1,f2,f3):
    ''' Load extracted data from files. Three files for xdata,ydata,weights.
    arguments: data directory, f1,f2,f3 
    returns : inpx,inpy,weights as arrays
    '''

    inpx=np.load(data_dir+f1+'.npy')
    inpy=np.load(data_dir+f2+'.npy')
    wts=np.load(data_dir+f3+'.npy')
    print(inpx.shape,inpy.shape)
    
    
    return inpx,inpy,wts


### Format data


In [7]:
#### Shuffle and split data ####

def f_shuffle_data(inpx,inpy,wts):
    ## Shuffle data
    
    # Setting seed
    seed=243
    np.random.seed(seed=seed)

    ## Get shuffled array of indices
    shuffle_arr=np.arange(inpx.shape[0])
    np.random.shuffle(shuffle_arr)
    inpx=inpx[shuffle_arr]
    inpy=inpy[shuffle_arr]
    wts=wts[shuffle_arr]

    return inpx,inpy,wts

def f_drop_data(inpx,inpy,wts,data_size):
    # Drop data for quick training. Just taking the slice of the data from the top.
    
    full_size=inpy.shape[0]
    assert(data_size<=full_size),"data_size: %s in f_drop_data is more than full data size: %s"%(data_size,full_size)
        
    temp=inpx[:data_size]
    del(inpx)
    inpx=temp.copy()
    temp=inpy[:data_size]
    del(inpy)
    inpy=temp.copy()
    temp=wts[:data_size]
    del(wts)
    wts=temp.copy()
    
    del(temp)    
    
    
    return (inpx,inpy,wts)        
        
       

def f_split_data(inpx,inpy,wts,test_fraction):
    '''
    Split data for training and test. validation from training piece of data.
    !! Warning this code deletes inpx,inpy inside the function. can't help it because the arrays are too big!!
    '''
    
    num=inpx.shape[0]
    test_idx=int(test_fraction*num)
    train_idx=num-test_idx

    train_x=inpx[:train_idx]
    train_y=inpy[:train_idx]
    train_wts=wts[:train_idx]
    
    test_x=inpx[train_idx:]
    test_y=inpy[train_idx:]
    test_wts=wts[train_idx:]
    
    return train_x,train_y,train_wts,test_x,test_y,test_wts


def f_format_data(inpx,inpy,wts,shuffle_flag=True,drop_data=True,data_size=1000,test_fraction=0.25):
    ''' Shuffle, drop and split data for train-test
    '''
    # Shuffle data
    if shuffle_flag: inpx,inpy,wts=f_shuffle_data(inpx,inpy,wts)
    # Drop data
    if drop_data: inpx,inpy,wts=f_drop_data(inpx,inpy,wts,data_size)

#     print(inpy[inpy==0.0].shape,inpy[inpy>0.0].shape,inpy.shape)
    
#     # Plot data
#     plt.figure()
#     plt.plot(inpy[:],linestyle='',marker='*',markersize=1)
#     plt.title("Plot of y data after shuffle")
#     plt.show() 
    
    # Split data into train-test.
    train_x,train_y,train_wts,test_x,test_y,test_wts=f_split_data(inpx,inpy,wts,test_fraction)
    
    print('Data sizes: train_x{0},train_y{1},test_x{2},test_y{3}'.format(train_x.shape,train_y.shape,test_x.shape,test_y.shape))

    return train_x,train_y,train_wts,test_x,test_y,test_wts


### Model details

In [19]:
### Defining all the models tried in the study

def f_define_model(inpx,name):
    '''
    Function that defines the model and compiles it.
    '''
    
    inputs = layers.Input(shape=inpx.shape[1:])
    h = inputs
    
    # Choose model
    if name=='1':
        print("model %s"%name)
        # Convolutional layers
        conv_sizes=[10, 10, 10]
        conv_args = dict(kernel_size=(3, 3, 3), activation='relu', padding='same')
        for conv_size in conv_sizes:
            h = layers.Conv3D(conv_size, **conv_args)(h)
            h = layers.MaxPooling3D(pool_size=(2, 2, 2))(h)
    #         h = layers.Dropout(0.5)(h)
        h = layers.Flatten()(h)

        # Fully connected  layers
        h = layers.Dense(10, activation='relu')(h)
        #    h = layers.Dropout(0.5)(h)

        # Ouptut layer
        outputs = layers.Dense(1, activation='sigmoid')(h)
    
    
    elif name=='2':
        print("model %s"%name)
        # Convolutional layers
        conv_sizes=[10,10,10]
        conv_args = dict(kernel_size=(3, 3, 3), activation='relu', padding='same')
        for conv_size in conv_sizes:
            h = layers.Conv3D(conv_size, **conv_args)(h)
            h = layers.MaxPooling3D(pool_size=(2, 2, 2))(h)
            h = layers.Dropout(0.5)(h)
        h = layers.Flatten()(h)

        # Fully connected  layers
        h = layers.Dense(64, activation='relu')(h)
        h = layers.Dropout(0.5)(h)

        # Ouptut layer
        outputs = layers.Dense(1, activation='sigmoid')(h)
        
    elif name=='3':
        print("model %s"%name)
        # Convolutional layers
        conv_sizes=[6,6,6]
        conv_args = dict(kernel_size=(3, 3, 3), activation='relu', padding='same')
        for conv_size in conv_sizes:
            h = layers.Conv3D(conv_size, **conv_args)(h)
            h = layers.MaxPooling3D(pool_size=(2, 2, 2))(h)
            h = layers.Dropout(0.5)(h)
        h = layers.Flatten()(h)

        # Fully connected  layers
        h = layers.Dense(64, activation='relu')(h)
        h = layers.Dropout(0.5)(h)

        # Ouptut layer
        outputs = layers.Dense(1, activation='sigmoid')(h)
    
    elif name=='4':
        print("model %s"%name)
        # Convolutional layers
        conv_sizes=[6,6,6]
        conv_args = dict(kernel_size=(3, 3, 3), activation='relu', padding='same')
        for conv_size in conv_sizes:
            h = layers.Conv3D(conv_size, **conv_args)(h)
            h = layers.MaxPooling3D(pool_size=(2, 2, 2))(h)
            h = layers.Dropout(0.5)(h)
        h = layers.Flatten()(h)

        # Fully connected  layers
        h = layers.Dense(120, activation='relu')(h)
        h = layers.Dropout(0.5)(h)

        # Ouptut layer
        outputs = layers.Dense(1, activation='sigmoid')(h)
        
    elif name=='5':
        print("model %s"%name)
        # Convolutional layers
        conv_sizes=[6,6]
        conv_args = dict(kernel_size=(2, 4, 15), activation='relu', padding='same')
        for conv_size in conv_sizes:
            h = layers.Conv3D(conv_size, **conv_args)(h)
            h = layers.MaxPooling3D(pool_size=(3, 3, 3))(h)
            h = layers.Dropout(0.5)(h)
        h = layers.Flatten()(h)

        # Fully connected  layers
        h = layers.Dense(120, activation='relu')(h)
        h = layers.Dropout(0.5)(h)

        # Ouptut layer
        outputs = layers.Dense(1, activation='sigmoid')(h)
        
    elif name=='resnet50':
        model=keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs, input_shape=inpx.shape[1:], pooling=max, classes=1000)
        
        return model
    
    ############################################
    ####### Compile model ######################
    ############################################
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#     model.summary()



    return model


### Train and perform fit

In [20]:

def f_train_model(model,inpx,inpy,num_epochs=5):
    '''
    Train model. Returns just history.history
    '''
    cv_fraction=0.33 # Fraction of data for cross validation
    
    history=model.fit(x=inpx, y=inpy,
                    batch_size=32,
                    epochs=num_epochs,
                    verbose=1,
#                     callbacks = [callbacks.ModelCheckpoint('./rpv_weights.h5')],
                    validation_split=cv_fraction,
                    shuffle=True
                )
    
    print("Number of parameters",model.count_params())
    
    return history.history

def f_plot_learning(history):
    
    fig=plt.figure()
    # Plot training & validation accuracy values
    fig.add_subplot(2,1,1)
    plt.plot(history['acc'],label='Train')
    plt.plot(history['val_acc'],label='Validation')
#     plt.title('Model accuracy')
    plt.ylabel('Accuracy')

    # Plot loss values
    fig.add_subplot(2,1,2)
    plt.plot(history['loss'],label='Train')
    plt.plot(history['val_loss'],label='Validation')
#     plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.show()

def f_plot_roc_curve(fpr,tpr):
    '''
    Module for roc plot and printing AUC
    '''
    plt.figure()
    # plt.plot(fpr,tpr)
    plt.scatter(fpr,tpr)
    plt.xscale('log')
    plt.show()

    # AUC 
    auc_val = auc(fpr, tpr)
    print("AUC: ",auc_val)
    

def f_test_model(xdata,ydata,wts,model,model_name,model_save_dir,test_status=False):
    '''
    Test model and make ROC plot
    If model has been tested, store the y-predict values
    and read them in next time.

    '''
    
    test_file_name=model_save_dir+'y-predict_model-'+str(model_name)+'.pred'
    
    
#     model.evaluate(xdata,ydata,sample_weights=wts,verbose=1)
    if not test_status:# Predict values and store to file.
        y_pred=model.predict(xdata,verbose=1)
        # Save prediction file
        np.savetxt(test_file_name,y_pred)
        
    else: # Load y_predictions from file.
        print("Using test prediction from previous test",test_file_name)
        y_pred=np.loadtxt(test_file_name)
        
#     print(y_pred)
    fpr,tpr,threshold=roc_curve(ydata,y_pred,sample_weight=wts)
    print(fpr.shape,tpr.shape,threshold.shape)
    f_plot_roc_curve(fpr,tpr)


def f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs,train_status=False,test_status=False):
    '''
    Compile, train, save and test the model.
    Steps:
    - Compile
    - Train
    - Save
    - Read
    - Plot
    - Test
    
    Note: Cross-validation data is built into the training. So, train_{x/y} contains the training and cval data.
    '''
    
    model_save_dir='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/saved_models/'
    model_name=model_dict['name'] # string for the model
    fname_model,fname_history='model_{0}.h5'.format(model_name),'history_{0}.pickle'.format(model_name)
    
    if not train_status: # If not trained before, train the model and save it.

        ########################
        # Compile model
        model=f_define_model(train_x,model_name)
        # Train model
        history=f_train_model(model,train_x,train_y,num_epochs)

        ########################
        # Save model and history
        model.save(model_save_dir+fname_model)
        with open(model_save_dir+fname_history, 'wb') as f:
                pickle.dump(history, f)
    
    else:
        print("Using trained model")

        
    ########################
    ### Read model and history
    
    ### Check if files exist
    assert os.path.exists(model_save_dir+fname_model),"Model not saved"
    assert os.path.exists(model_save_dir+fname_history),"History not saved"
    
    model=load_model(model_save_dir+fname_model)
    with open(model_save_dir+fname_history,'rb') as f:
        history= pickle.load(f)
    
    ########################
    model.summary()
    # Plot tested model
    f_plot_learning(history)
    
    ########################
    # Test model
    f_test_model(test_x,test_y,test_wts,model,model_dict['name'],model_save_dir,test_status)

    model_dict['model'],model_dict['history']=model,history
    
    return model_dict



## Execution starts

### Get Data

In [ ]:
train_status=True
# train_status=False
# test_status=False
test_status=True


model_dict={'name':'1','description':'simplest','model':None,'history':None}
model_dict1=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status,test_status)



In [16]:
if __name__=='__main__':
    ###Extract data
    data_dir='/global/project/projectdirs/dasrepo/vpa/ice_cube/data_for_cnn/extracted_data_v/data/'
    f1,f2,f3='processed_input_regular_x','processed_input_regular_y','processed_input_regular_wts'
    inpx,inpy,wts=f_load_data(data_dir,f1,f2,f3)
    # print(sys.getsizeof(inpx))

    ###Format data
    ### cross-validation done with part of train data.
    train_x,train_y,train_wts,test_x,test_y,test_wts=f_format_data(inpx,inpy,wts,shuffle_flag=True,drop_data=False,data_size=10000,test_fraction=0.25)

    del(inpx,inpy,wts)

(136066, 10, 20, 60, 1) (136066,)
Data sizes: train_x(102050, 10, 20, 60, 1),train_y(102050,),test_x(34016, 10, 20, 60, 1),test_y(34016,)


In [21]:
# train_status=True
train_status=False
test_status=False
# test_status=True


model_dict={'name':'resnet50','description':'simplest','model':None,'history':None}
model_dict1=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status=train_status,test_status=test_status)


ValueError: `input_shape` must be a tuple of three integers.

### Model 1

### Model 2

In [10]:
# train_status=True
train_status=False
test_status=False
# test_status=True

model_dict={'name':'2','description':'simplest','model':None,'history':None}
model_dict2=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status,test_status)


model 2
Train on 68373 samples, validate on 33677 samples
Epoch 1/5
68373/68373 [==============================] - 427s 6ms/step - loss: 1.2012 - acc: 0.7939 - val_loss: 0.4874 - val_acc: 0.8059
Epoch 2/5
68373/68373 [==============================] - 432s 6ms/step - loss: 0.4733 - acc: 0.8139 - val_loss: 0.4864 - val_acc: 0.8064
Epoch 3/5
68373/68373 [==============================] - 426s 6ms/step - loss: 0.4457 - acc: 0.8226 - val_loss: 0.4683 - val_acc: 0.8077
Epoch 4/5
68373/68373 [==============================] - 410s 6ms/step - loss: 0.4177 - acc: 0.8341 - val_loss: 0.4411 - val_acc: 0.8115
Epoch 5/5
68373/68373 [==============================] - 408s 6ms/step - loss: 0.3937 - acc: 0.8464 - val_loss: 0.4181 - val_acc: 0.8176
Number of parameters 14789
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 20, 60, 1)     0         
________________________________

FigureCanvasNbAgg()

FigureCanvasNbAgg()

34016/34016 [==============================] - 77s 2ms/step
(33911,) (33911,) (33911,)


FigureCanvasNbAgg()

AUC:  0.842717218514279


### Model 3

In [11]:
# train_status=True
train_status=False
test_status=False
# test_status=True

model_dict={'name':'3','description':'6conv size, extra dropout','model':None,'history':None}
model_dict3=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status,test_status)



model 3
Train on 68373 samples, validate on 33677 samples
Epoch 1/5
68373/68373 [==============================] - 257s 4ms/step - loss: 1.3657 - acc: 0.7929 - val_loss: 0.4903 - val_acc: 0.8057
Epoch 2/5
68373/68373 [==============================] - 260s 4ms/step - loss: 0.4802 - acc: 0.8090 - val_loss: 0.4827 - val_acc: 0.8060
Epoch 3/5
68373/68373 [==============================] - 264s 4ms/step - loss: 0.4656 - acc: 0.8130 - val_loss: 0.4758 - val_acc: 0.8065
Epoch 4/5
68373/68373 [==============================] - 248s 4ms/step - loss: 0.4438 - acc: 0.8212 - val_loss: 0.4602 - val_acc: 0.8089
Epoch 5/5
68373/68373 [==============================] - 262s 4ms/step - loss: 0.4067 - acc: 0.8360 - val_loss: 0.4425 - val_acc: 0.8134
Number of parameters 7629
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10, 20, 60, 1)     0         
_________________________________

FigureCanvasNbAgg()

FigureCanvasNbAgg()

34016/34016 [==============================] - 55s 2ms/step
(33888,) (33888,) (33888,)


FigureCanvasNbAgg()

AUC:  0.8645203695240572


### Model 4

In [12]:
# train_status=True
train_status=False
test_status=False
# test_status=True

model_dict={'name':'4','description':'6conv size, extra dropout','model':None,'history':None}
model_dict4=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status,test_status)




model 4
Train on 68373 samples, validate on 33677 samples
Epoch 1/5
68373/68373 [==============================] - 268s 4ms/step - loss: 1.3012 - acc: 0.7950 - val_loss: 0.4926 - val_acc: 0.8057
Epoch 2/5
68373/68373 [==============================] - 245s 4ms/step - loss: 0.4812 - acc: 0.8102 - val_loss: 0.4772 - val_acc: 0.8065
Epoch 3/5
68373/68373 [==============================] - 248s 4ms/step - loss: 0.4420 - acc: 0.8204 - val_loss: 0.3991 - val_acc: 0.8149
Epoch 4/5
68373/68373 [==============================] - 259s 4ms/step - loss: 0.3957 - acc: 0.8365 - val_loss: 0.3577 - val_acc: 0.8298
Epoch 5/5
68373/68373 [==============================] - 262s 4ms/step - loss: 0.3735 - acc: 0.8464 - val_loss: 0.3495 - val_acc: 0.8316
Number of parameters 12445
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10, 20, 60, 1)     0         
________________________________

FigureCanvasNbAgg()

FigureCanvasNbAgg()

34016/34016 [==============================] - 53s 2ms/step
(33315,) (33315,) (33315,)


FigureCanvasNbAgg()

AUC:  0.8858907324834038


### Model 5

In [13]:
# train_status=True
train_status=False
test_status=False
# test_status=True

model_dict={'name':'5','description':' 2conv layers (6), bigger Maxpool size','model':None,'history':None}
model_dict5=f_perform_fit(train_x,train_y,train_wts,test_x,test_y,test_wts,model_dict,num_epochs=5,train_status,test_status)


model 5
Train on 68373 samples, validate on 33677 samples
Epoch 1/5
68373/68373 [==============================] - 550s 8ms/step - loss: 3.0213 - acc: 0.8055 - val_loss: 2.9154 - val_acc: 0.8065
Epoch 2/5
68373/68373 [==============================] - 550s 8ms/step - loss: 0.9137 - acc: 0.8043 - val_loss: 0.4864 - val_acc: 0.8069
Epoch 3/5
68373/68373 [==============================] - 560s 8ms/step - loss: 0.4873 - acc: 0.8084 - val_loss: 0.4835 - val_acc: 0.8067
Epoch 4/5
68373/68373 [==============================] - 567s 8ms/step - loss: 0.4853 - acc: 0.8085 - val_loss: 0.4810 - val_acc: 0.8061
Epoch 5/5
68373/68373 [==============================] - 555s 8ms/step - loss: 0.4801 - acc: 0.8092 - val_loss: 0.4812 - val_acc: 0.8061
Number of parameters 13933
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10, 20, 60, 1)     0         
________________________________

FigureCanvasNbAgg()

FigureCanvasNbAgg()

 6272/34016 [====>.........................] - ETA: 1:26

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Comparing models

In [14]:
### Comparing different models:

for num,md in enumerate([model_dict1,model_dict2,model_dict3,model_dict4,model_dict5]):
    hist=md
#     print(md)
    print('Model %s'%(num+1))
    for key in hist.keys():
        print(key,hist[key])
        

Model 1
name 1
description simplest
model <keras.engine.training.Model object at 0x2aed939d4860>
history {'val_loss': [0.3614254333144126, 0.3227415867653143, 0.26999167760410264, 0.24859269623816302, 0.2539909265686087], 'val_acc': [0.8614781601722009, 0.8775722303090896, 0.9077411883516705, 0.9123140422282034, 0.9008819075368711], 'loss': [0.5055176187268231, 0.33989128664735485, 0.28450851341929856, 0.26161940508354103, 0.2418150477575659], 'acc': [0.8289529492653417, 0.8804645108466677, 0.9014815789867288, 0.9083848887736271, 0.913811007269823]}
Model 2
name 2
description simplest
model <keras.engine.training.Model object at 0x2aed9dbbb208>
history {'val_loss': [0.48737753331298395, 0.48636280836439416, 0.4682738286653423, 0.4410965909961992, 0.4180907299727999], 'val_acc': [0.8058615672452775, 0.8063663628030766, 0.807672892482086, 0.8114737060937497, 0.81756094664368], 'loss': [1.2012339293326477, 0.4733139974947547, 0.4456613794283641, 0.41768144707070703, 0.3936835106274213], '

In [15]:
# model_dict1

### Re-plot

In [16]:
# # Re-plot
# m=model_dict1
# f_plot_learning(m['history'])
# f_test_model(test_x,test_y,m['model'])

## -----------------------------------------------

### Questions:
- Why are fpr and tpr different for 2 different models?


#### Notes:
- model.fit 
    - batch_size= sample of data used for training (subset of full training set). 
    - epoch= number of runs over training data
    - callbacks=
    
- for layers.Input need size (x,y,z,1) in channels_last mode.

#### Roc curve notes:
- We know y-value depending on signal or background (0 or 1).
- The 3D-Cnn gives us a prediction for y, as a float between 0 or 1.
- We must use a cut (threshold) to determine what constitues 0 / 1. Eg. 0.5
- This gives us a false +ve rate a, true +ve .(fpr and tpr)
- Roc curve plots this when varying the threshold
- AUC gives area under this curve.

In [17]:
# Plotting weights
print(train_wts.shape,test_wts.shape)

# Train data 
plt.figure()
plt.plot(train_wts)
plt.title("train + cv data weigts ")
plt.show()

plt.figure()
plt.plot(test_wts)
plt.title("test data weights")
plt.show()



(102050,) (34016,)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

## To do
- use weights to split dataset into train- test
- plot on log scale
- pick the best model
- test on reserve data set
- running with multiple cores on a batch node.
    - ensure you're running inside your conda environment. 
- using multiple nodes
- using GPU nodes
- Test a host of models using ipyparallel
- make changes to incorporate regular data in training and reserved data in testing
- way to store tested values for easy plotting